In [ ]:
import functools
import numpy as np
import matplotlib.pyplot as plt
import scipy as sp
import tensorflow.compat.v1 as tf

tf.disable_eager_execution()

# N-S Solver

## Boundary Condition

In [ ]:
def update_boundary(f, order, type='periodic'):
  """Update the boundary of `f` with width `order`."""

  def periodic():
    g = f[order:-order, order:-order, order:-order]
    g = tf.concat([g[:, :, -order:], g, g[:, :, :order]], 2)
    g = tf.concat([g[:, -order:, :], g, g[:, :order, :]], 1)
    g = tf.concat([g[-order:, :, :], g, g[:order, :, :]], 0)
    return g

  def homogeneous():
    g = f[order:-order, order:-order, order:-order]
    return tf.pad(g, 
                  ((order, order), (order, order), (order, order)), 'CONSTANT')

  if type == 'periodic':
    return periodic()
  elif type == 'homogeneous':
    return homogeneous()

## Operators

In [ ]:
#@title Space
_WEIGHTS = {
    # The backward sum of 2 nodes.
    'sum-': ([1.0, 1.0], 1.0),
    # The backward difference scheme for first order derivative.
    'ddx1-': ([-1.0, 1.0], 1.0),
    # The forward difference scheme for first order derivative.
    'ddx1+': ([0.0, -1.0, 1.0], 1.0),
    # The second order central difference scheme for first order derivative.
    'ddx2': ([-1., 0., 1.], 2.),
    # The fourth order central difference scheme for first order derivative.
    'ddx4': ([1., -8., 0., 8., -1.], 12.),
    # The second order central difference scheme for fourth order derivative.
    'd4x2': ([1., -4., 6., -4., 1.], None)
}


def apply_kernel(f, dim, scheme):
  """Apply weights to `f` corresponds to `scheme`."""
  (weights, _) = _WEIGHTS[scheme]
  nbc = len(weights) // 2

  ndim = f.get_shape().as_list()
  ndim = [ndim_i - 2 * nbc for ndim_i in ndim]

  if dim == 'x':
    g = [f[i:i + ndim[0], nbc:-nbc, nbc:-nbc] for i in range(len(weights))]
  elif dim == 'y':
    g = [f[nbc:-nbc, i:i + ndim[1], nbc:-nbc] for i in range(len(weights))]
  else:  # dim == 'z'
    g = [f[nbc:-nbc, nbc:-nbc, i:i + ndim[2]] for i in range(len(weights))]

  output = tf.zeros(ndim, dtype=tf.float32)
  for i in range(len(weights)):
    output += weights[i] * g[i]

  # Patch zeros to the boundary.
  return tf.pad(output, ((nbc, nbc), (nbc, nbc), (nbc, nbc)), 'CONSTANT')


def grad(f, h, dim, scheme):
  """Computes the gradient of `f` using `scheme`.

  The boundary condition of `f` has to be updated before calling `grad`. The
  returned tensor does NOT contain the boundary elements.
  """
  (_, factor) = _WEIGHTS[scheme]

  output = apply_kernel(f, dim, scheme)
  return output / (factor * h)


def grad_f(f, h, dim):
  """Computes the gradient of `f` on faces."""
  if dim == 'x':
    return (f[1:, :, :] - f[:-1, :, :]) / h
  elif dim == 'y':
    return (f[:, 1:, :] - f[:, :-1, :]) / h
  else:  # dim == 'z'
    return (f[:, :, 1:] - f[:, :, :-1]) / h


def quick(f, dim):
  """Computes the scalar value on the cell face using the QUICK scheme.

  The boundary condition of `f` has to be updated before calling `quick`. The
  returned tensor does NOT contain the boundary elements.
  """
  if dim == 'x':
    f_ww = f[:-3, 2:-2, 2:-2]
    f_w = f[1:-2, 2:-2, 2:-2]
    f_p = f[2:-1, 2:-2, 2:-2]
    f_e = f[3:, 2:-2, 2:-2]
  elif dim == 'y':
    f_ww = f[2:-2, :-3, 2:-2]
    f_w = f[2:-2, 1:-2, 2:-2]
    f_p = f[2:-2, 2:-1, 2:-2]
    f_e = f[2:-2, 3:, 2:-2]
  else:  # dim == 'z'
    f_ww = f[2:-2, 2:-2, :-3]
    f_w = f[2:-2, 2:-2, 1:-2]
    f_p = f[2:-2, 2:-2, 2:-1]
    f_e = f[2:-2, 2:-2, 3:]

  f_pos = 0.75 * f_w + 0.375 * f_p - 0.125 * f_ww
  f_neg = 0.75 * f_p + 0.375 * f_w - 0.125 * f_e

  return (f_pos, f_neg)


def rhou_face(rhou, p, params, dim):
  if dim == 'x':
    rhou_w = rhou[1:-2, 2:-2, 2:-2]
    rhou_e = rhou[2:-1, 2:-2, 2:-2]
    p_ww = p[:-3, 2:-2, 2:-2]
    p_w = p[1:-2, 2:-2, 2:-2]
    p_p = p[2:-1, 2:-2, 2:-2]
    p_e = p[3:, 2:-2, 2:-2]
  elif dim == 'y':
    rhou_w = rhou[2:-2, 1:-2, 2:-2]
    rhou_e = rhou[2:-2, 2:-1, 2:-2]
    p_ww = p[2:-2, :-3, 2:-2]
    p_w = p[2:-2, 1:-2, 2:-2]
    p_p = p[2:-2, 2:-1, 2:-2]
    p_e = p[2:-2, 3:, 2:-2]
  else:  # dim == 'z'
    rhou_w = rhou[2:-2, 2:-2, 1:-2]
    rhou_e = rhou[2:-2, 2:-2, 2:-1]
    p_ww = p[2:-2, 2:-2, :-3]
    p_w = p[2:-2, 2:-2, 1:-2]
    p_p = p[2:-2, 2:-2, 2:-1]
    p_e = p[2:-2, 2:-2, 3:]

  return (0.5 * (rhou_w + rhou_e) - (params['dt'] / 4.0 / params['dx']) *
          (-p_ww + 3. * p_w - 3. * p_p + p_e))


def flux_quick(sc, rhou_o, p, params, dim):
  sc_x_pos, sc_x_neg = quick(sc, dim)

  rhou = rhou_face(rhou_o, p, params, dim)

  return (0.5 * (rhou + tf.abs(rhou)) * sc_x_pos + 0.5 *
          (rhou - tf.abs(rhou)) * sc_x_neg)

In [ ]:
#@title Time
def explicit_iterative_time_advancement(state, update, dt):
  return {varname: value_old + dt * update[varname]
          for varname, value_old in state.items()}


In [ ]:
#@title Filter
def filter_3d(f, nfilter):

  cond = lambda i, fr: i < nfilter
  
  def body(i, fr):
    buf = tf.expand_dims(fr, 0)
    buf = tf.expand_dims(buf, -1)
    kernel = tf.ones((3, 3, 3, 1, 1), dtype=tf.float32) / 27.
    return (i + 1, 
            update_boundary(
                tf.squeeze(tf.nn.conv3d(buf, kernel, (1, 1, 1, 1, 1), "SAME")), 
                1))

  i0 = tf.constant(0)
  _, fr = tf.while_loop(cond, body, (i0, f), 
                        back_prop=False, 
                        return_same_structure=True)

  return fr

## Equations

In [ ]:
#@title Momentum
def implicit_solve_momentum(momentum_new, state, state_m, jacobian, params):
  """Solves the momentum equation implicitly for time advancement.

  The linear system is like 
  (I - J_x / rho)(I - J_y / rho)(u - u_k) = rhs.
  We compose the matrix to be inverted on the left hand side 
  and then solve the linear system. 

  Args:
    momentum_new: A dictionary holding values for updated values on rhs.
    state: The latest state from subiterations.
    state_m: A dictionary holding values at mid time step for varables.
    jacobian: A dictionary holding Jacobian matrix for each velocity component.
    params: Parameters specifying the fluid properties and computational setup.

  Returns:
    The updated momentum variables.
  """
  nx, ny, nz = momentum_new['rhou'].get_shape().as_list()
  momentum = {key: momentum_new[key] for key in momentum_new}

  # Ensamble Jacobian matrices A_x = I - j_x / rho_t; A_y = I - j_y / rho_t.
  # Put the Jacobian direction axis to the last component to
  # fit into tf.linalg.solve. 
  eye_x = tf.broadcast_to(tf.eye(nx), [nz, ny, nx, nx])
  eye_y = tf.broadcast_to(tf.eye(ny), [nz, nx, ny, ny])
  rho_t = tf.transpose(state_m['rho'], perm=[2, 1, 0])  # dimention (z, y, x)
  j_x = jacobian['a_diff'][0]
  j_x = tf.transpose(j_x, perm=[3, 2, 0, 1])
  j_x = j_x / tf.expand_dims(rho_t, -1)
  rho_t = tf.transpose(state_m['rho'], perm=[2, 0, 1])  # (z, x, y)
  j_y = tf.transpose(jacobian['a_diff'][1], perm=[3, 0, 1, 2])  #(z, x, y, y)
  j_y = j_y / tf.expand_dims(rho_t, -1)
  rho_vel = {}
  rho_vel['u'] = momentum_new['rhou']
  rho_vel['v'] = momentum_new['rhov']
  diff_coeff = {}
  diff_coeff['u'] = [4./3., 1]
  diff_coeff['v'] = [1, 4./3.]

  # Construct and solve linear system.
  for key in rho_vel:
    aa_x = eye_x - diff_coeff[key][0] * j_x
    aa_y = eye_y - diff_coeff[key][1] * j_y
    rhs = rho_vel[key] / state_m['rho'] - state[key]
    # in axis (z, y, x)
    z = tf.linalg.solve(aa_x, tf.expand_dims(tf.transpose(rhs), -1))
    # Permutes z to conform with J_y direction.
    z = tf.transpose(tf.squeeze(z), perm=[0, 2, 1])  # (z, x, y)
    x = tf.linalg.solve(aa_y, tf.expand_dims(z, -1))
    vel = tf.transpose(tf.squeeze(x), perm=[1, 2, 0]) + state[key]  # (x, y, z)
    momentum['rho{}'.format(key)] = state_m['rho'] * vel
  return momentum

def shear_stress(u, v, w, rho, params):
  """Compute the viscous stress tensor."""
  du_11 = grad(u, params['dx'], 'x', 'ddx2')
  du_12 = grad(u, params['dy'], 'y', 'ddx2')
  du_13 = grad(u, params['dz'], 'z', 'ddx2')
  du_21 = grad(v, params['dx'], 'x', 'ddx2')
  du_22 = grad(v, params['dy'], 'y', 'ddx2')
  du_23 = grad(v, params['dz'], 'z', 'ddx2')
  du_31 = grad(w, params['dx'], 'x', 'ddx2')
  du_32 = grad(w, params['dy'], 'y', 'ddx2')
  du_33 = grad(w, params['dz'], 'z', 'ddx2')

  S11 = du_11
  S12 = 0.5 * (du_12 + du_21)
  S13 = 0.5 * (du_13 + du_31)
  S21 = S12
  S22 = du_22
  S23 = 0.5 * (du_23 + du_32)
  S31 = S13
  S32 = S23
  S33 = du_33

  du_ii = du_11 + du_22 + du_33
  tau11 = 2.0 * rho * params['nu'] * (S11 - 1.0 / 3.0 * du_ii)
  tau12 = 2.0 * rho * params['nu'] * S12
  tau13 = 2.0 * rho * params['nu'] * S13
  tau21 = 2.0 * rho * params['nu'] * S21
  tau22 = 2.0 * rho * params['nu'] * (S22 - 1.0 / 3.0 * du_ii)
  tau23 = 2.0 * rho * params['nu'] * S23
  tau31 = 2.0 * rho * params['nu'] * S31
  tau32 = 2.0 * rho * params['nu'] * S32
  tau33 = 2.0 * rho * params['nu'] * (S33 - 1.0 / 3.0 * du_ii)

  # Update boundary conditions for the shear stresses.
  output = {
      'xx': tau11,
      'xy': tau12,
      'xz': tau13,
      'yx': tau21,
      'yy': tau22,
      'yz': tau23,
      'zx': tau31,
      'zy': tau32,
      'zz': tau33
  }
  return {
      key: update_boundary(value, params['nbc'])
      for key, value in output.items()
  }


def get_diffusion(u, v, w, rho, params):
  """Computes the diffusion term for the three momentum equations."""
  ddx = functools.partial(grad, h=params['dx'], dim='x', scheme='ddx2')
  ddy = functools.partial(grad, h=params['dy'], dim='y', scheme='ddx2')
  ddz = functools.partial(grad, h=params['dz'], dim='z', scheme='ddx2')

  tau = shear_stress(u, v, w, rho, params)

  return {
      'rhou': ddx(tau['xx']) + ddy(tau['xy']) + ddz(tau['xz']),
      'rhov': ddx(tau['yx']) + ddy(tau['yy']) + ddz(tau['yz']),
      'rhow': ddx(tau['zx']) + ddy(tau['zy']) + ddz(tau['zz']),
  }


def get_diffusion_stencil_3(u, v, w, rho, params):
  """Computes the diffusion terms with three-node stencils."""
  dims = ('x', 'y', 'z')
  velocity = {'x': u, 'y': v, 'z': w}

  # Define operators that computes the central difference gradient.
  grad_c = lambda dim: functools.partial(
      grad, h=params['d{}'.format(dim)], dim=dim, scheme='ddx2')

  # Define operators that comptues the backward summation.
  sum_b = lambda dim: functools.partial(apply_kernel, dim=dim, scheme='sum-')

  # Defines operators that comptues the backward difference.
  diff_b = lambda dim: functools.partial(apply_kernel, dim=dim, scheme='ddx1-')

  # Defines operators that computes gradient from faces to nodes.
  diff_p = lambda dim: functools.partial(apply_kernel, dim=dim, scheme='ddx1+')

  # Compute the dynamic viscosity.
  mu = rho * params['nu']

  # Compute the dynamic viscosity on different faces.
  mu_f = {dim: 0.5 * sum_b(dim)(mu) for dim in dims}

  # Compute the velocity gradients on faces.
  dudx_f = {
      k: {dim: diff_b(dim)(val) / params['d{}'.format(dim)] for dim in dims
         } for k, val in velocity.items()
  }

  # Compute the velocity gradient on nodes with second order central difference.
  dudx_c = {
      k: {dim: grad_c(dim)(val) for dim in dims} for k, val in velocity.items()
  }

  def tangential_diffusion_fn(dim):
    """Computes the diffusion term along direction of a vlocity component."""
    dims_n = list(dims)
    dims_n.remove(dim)

    h = params['d{}'.format(dim)]

    output = diff_p(dim)(4.0 / 3.0 * mu_f[dim] * dudx_f[dim][dim]) / h
    for i in dims_n:
      output -= grad_c(dim)(2.0 / 3.0 * mu * dudx_c[i][i])

    return output

  def normal_diffusion_fn(dim, dim_n):
    """Computes the diffusion term normal to a velocity component."""
    h = params['d{}'.format(dim_n)]
    return (diff_p(dim_n)(mu_f[dim_n] * dudx_f[dim][dim_n]) / h +
            grad_c(dim_n)(mu * dudx_c[dim_n][dim]))

  def diffusion_fn(dim):
    """Computes the sum of diffusion terms of velocity component `dim`."""
    output = tangential_diffusion_fn(dim)
    for i in dims:
      if i == dim:
        continue
      output += normal_diffusion_fn(dim, i)
    return output

  return {
      'rhou': diffusion_fn('x'),
      'rhov': diffusion_fn('y'),
      'rhow': diffusion_fn('z'),
  }

def momentum(state_old, state, params):
  """Time advancement of the momentum.

  Advance the momentum in conservative form, i.e. rhou, rhov, and rhow.

  Args:
    state_old: The starting state at the current time step.
    state: The latest state from subiterations.
    params: Parameters specifying the fluid properties and computational setup.

  Returns:
    The updated primitive state of u, v, and w for the next subiterations.
  """

  ddx = functools.partial(grad, h=params['dx'], dim='x', scheme='ddx2')
  ddy = functools.partial(grad, h=params['dy'], dim='y', scheme='ddx2')
  ddz = functools.partial(grad, h=params['dz'], dim='z', scheme='ddx2')

  def rhs(rhou, rhov, rhow, u, v, w, p, rho):
    """Computes the right hand side of the momentum equations."""

    def get_convection(f):
      """Computes the convection term for component `f`."""
      f_conv_x = flux_quick(f, rhou, p, params, 'x')
      f_conv_y = flux_quick(f, rhov, p, params, 'y')
      f_conv_z = flux_quick(f, rhow, p, params, 'z')

      conv_x = grad_f(f_conv_x, params['dx'], 'x')
      conv_y = grad_f(f_conv_y, params['dy'], 'y')
      conv_z = grad_f(f_conv_z, params['dz'], 'z')

      return tf.pad(conv_x + conv_y + conv_z, ((2, 2), (2, 2), (2, 2)),
                    'CONSTANT')

    # Convection terms.
    conv = {
        'rhou': get_convection(u),
        'rhov': get_convection(v),
        'rhow': get_convection(w),
    }

    # Diffusion terms.
    diff = get_diffusion_stencil_3(u, v, w, rho, params)

    # Momentum x.
    rhs_rhou = -conv['rhou'] + diff['rhou'] - ddx(p)

    # Momentum y.
    rhs_rhov = -conv['rhov'] + diff['rhov'] - ddy(p)

    # Momentum z.
    rhs_rhow = -conv['rhow'] + diff['rhow'] - ddz(p)

    return {'rhou': rhs_rhou, 'rhov': rhs_rhov, 'rhow': rhs_rhow}

  def build_jacobian_momentum(rho, params):
    """Computes the Jacobian for the momentum equation."""
    ndim = rho.get_shape().as_list()
    nbc = params['nbc']
    # Permutes direction to (x, y, z), (y, z, x).
    perm = [[0, 1, 2], [1, 2, 0]]
    perm_end = [[0, 1, 2, 3], [3, 0, 1, 2]]
    grid = [params['dx'], params['dy']]
    a_diff = [None] * (len(ndim) - 1)
    for i in range(len(ndim) - 1):
      rho_t = tf.transpose(rho, perm=perm[i])
      drho = [None] * 3
      # Calculate the coefficients of the velocity component in diffusion terms.
      drho[0] = (rho_t[2:-2, 2:-2, 2:-2] + rho_t[3:-1, 2:-2, 2:-2])
      drho[1] = - (rho_t[1:-3, 2:-2, 2:-2] +
                   2 * rho_t[2:-2, 2:-2, 2:-2] + rho_t[3:-1, 2:-2, 2:-2])
      drho[2] = (rho_t[1:-3, 2:-2, 2:-2] + rho_t[2:-2, 2:-2, 2:-2])
      buf = params['dt'] * params['nu'] * np.array(drho) / 4. / grid[i]**2
      # Calculates the subdiagonal vectors for the Jacobian matrix.
      diagonals_0 = tf.pad(buf[0], ([1, 1], [0, 0], [0, 0]), 'constant')
      diagonals_1 = tf.pad(buf[1], ([1, 1], [0, 0], [0, 0]), 'constant')
      diagonals_2 = tf.pad(buf[2], ([2, 0], [0, 0], [0, 0]), 'constant')

      # Uses default permutation (x, y, z) -> (z, y, x) to conform with tf.stack
      # on stacking on the last dimension.
      diagonals = tf.stack([tf.transpose(diagonals_0),
                            tf.transpose(diagonals_1),
                            tf.transpose(diagonals_2)], axis=-1)
      diagonals = tf.transpose(diagonals, perm=[0, 1, 3, 2])

      # Ensembles tridiagonal Jacobian matrices.
      a_diff[i] = tf.linalg.diag(diagonals, k=(-1, 1))
      a_diff[i] = tf.transpose(a_diff[i], perm=[2, 3, 1, 0])
      a_diff[i] = tf.pad(a_diff[i], ([1,1], [1, 1], [nbc, nbc], [nbc, nbc]),
                         'constant')
      a_diff[i] = tf.transpose(a_diff[i], perm=perm_end[i])
    return {'a_diff' : a_diff}

  # Update the momentum.
  momentum_old = {
      'rhou': state_old['rhou'],
      'rhov': state_old['rhov'],
      'rhow': state_old['rhow']
  }

  state_m = {}
  for varname in ('rhou', 'rhov', 'rhow', 'u', 'v', 'w', 'rho'):
    state_m.update({varname: 0.5 * (state_old[varname] + state[varname])})

  momentum_updates = rhs(state_m['rhou'], state_m['rhov'], state_m['rhow'],
                         state_m['u'], state_m['v'], state_m['w'], state['p'],
                         state_old['rho'])

  momentum_new = explicit_iterative_time_advancement(momentum_old,
                                                     momentum_updates,
                                                     params['dt'])
  if params['add_jacobian']:
    # Add Jacobian matrix
    jacobian = build_jacobian_momentum(state_m['rho'], params)
    # Add Jacobian to solve momentum equation
    momentum_new = implicit_solve_momentum(momentum_new, state, state_m, jacobian, params)
  # Update the boundary conditions for the new velocity and return.
  u_new = update_boundary(momentum_new['rhou'] / state_m['rho'], params['nbc'])
  v_new = update_boundary(momentum_new['rhov'] / state_m['rho'], params['nbc'])
  w_new = update_boundary(momentum_new['rhow'] / state_m['rho'], params['nbc'])

  return {
      'u': u_new,
      'v': v_new,
      'w': w_new,
      'rhou': state_m['rho'] * u_new,
      'rhov': state_m['rho'] * v_new,
      'rhow': state_m['rho'] * w_new
  }

In [ ]:
#@title Density
def density(state, params):
  """Updates the density based on the scalar information."""
  def regularize_scalar(sc):
    return tf.minimum(tf.maximum(sc, tf.zeros_like(sc)), tf.ones_like(sc))

  sc_buf = {sc_name: regularize_scalar(sc_val) for
            sc_name, sc_val in state.items()}
  sc_all = tf.convert_to_tensor([sc_val for sc_val in sc_buf.values()])
  sc_sum = tf.reduce_sum(sc_all, axis=0)
  y_n_buf = tf.ones_like(sc_sum) - sc_sum
  sc_buf.update({'YN': regularize_scalar(y_n_buf)})

  sc_total = tf.reduce_sum([sc_val for _, sc_val in sc_buf.items()], axis=0)
  sc_reg = {sc_name: sc_val / sc_total for sc_name, sc_val in sc_buf.items()}

  return (tf.reduce_sum(tf.convert_to_tensor([sc_val * params['rho'][sc_name] 
                         for sc_name, sc_val in sc_buf.items()]), axis=0))
                        

def drho(rho, rhoold, nfilter):
  return filter_3d(rho - rhoold, nfilter)

In [ ]:
#@title Scalar Transport

_NP_DTYPE = np.float32

def solve_scalar(rhs, state, jacobian, params):
  """Solves the mixing scalar equation.

  The linear system is like 
  (I - J_x / rho)(I - J_y / rho)(phi-phi_k) = rhs.
  We compose the matrix to be inverted on the left hand side 
  and then solve the linear system. 

  Args:
    rhs: A dictionary for the rhs.
    state: The latest state from subiterations.
    jacobian: A dictionary holding Jacobian matrix for each velocity component.
    params: Parameters specifying the fluid properties and computational setup.

  Returns:
    The updated scalar variables.
  """
  sc_names = params['sc_names']
  ones = tf.ones_like(state['rho'])
  # Permute to (z, y, x) to conform with tf.linalg.tridiagonal_solve.
  eye_x = tf.linalg.diag(tf.transpose(ones), k=0)
  # Permute to (z, x, y) to conform with tf.linalg.tridiagonal_solve.
  eye_y = tf.linalg.diag(tf.transpose(ones, perm=[2, 0, 1]))
  rhs_over_rho = {sc_name: rhs['rho_{}'.format(sc_name)]/state['rho']
                  for sc_name in sc_names} 
  sc = {}
  # Solves  A_x A_y x = b.
  for sc_name in sc_names:
    A_x = eye_x - tf.transpose(jacobian[sc_name][0], perm=[3, 2, 0, 1])
    z = tf.linalg.tridiagonal_solve(A_x, tf.transpose(rhs_over_rho[sc_name]
                                   - state[sc_name]), diagonals_format='matrix')
    # Permute to (x, y, z).
    z = tf.transpose(z)
    A_y = eye_y - tf.transpose(jacobian[sc_name][1], perm=[3, 0, 1, 2])
    x = tf.linalg.tridiagonal_solve(A_y, tf.transpose(z, perm=[2, 0, 1]),
                                    diagonals_format='matrix')
    sc[sc_name] = tf.transpose(x, perm=[1, 2, 0]) + state[sc_name]
  return sc

def rho_scalar(state_old, state, params):
  """Time advancement of the scalar transport equations."""
  # Define operators that comptues the backward summation.
  sx = functools.partial(apply_kernel, dim='x', scheme='sum-')
  sy = functools.partial(apply_kernel, dim='y', scheme='sum-')
  sz = functools.partial(apply_kernel, dim='z', scheme='sum-')

  # Defines operators that comptues the backward difference.
  fx = functools.partial(apply_kernel, dim='x', scheme='ddx1-')
  fy = functools.partial(apply_kernel, dim='y', scheme='ddx1-')
  fz = functools.partial(apply_kernel, dim='z', scheme='ddx1-')

  def rhs(rho, rhou_o, rhov_o, rhow_o, sc, sc_name):
    """Computes the right hand side of the scalar equations."""
    rho_x = 0.5 * sx(rho)[2:-1, 2:-2, 2:-2]
    rho_y = 0.5 * sy(rho)[2:-2, 2:-1, 2:-2]
    rho_z = 0.5 * sy(rho)[2:-2, 2:-2, 2:-1]

    # Convection.
    f_conv_x = flux_quick(sc, rhou_o, state['p'], params, 'x')
    f_conv_y = flux_quick(sc, rhov_o, state['p'], params, 'y')
    f_conv_z = flux_quick(sc, rhow_o, state['p'], params, 'z')
  
    conv_x = grad_f(f_conv_x, params['dx'], 'x')
    conv_y = grad_f(f_conv_y, params['dy'], 'y')
    conv_z = grad_f(f_conv_z, params['dz'], 'z')

    # Diffusion.
    f_diff_x = (
        rho_x * params['alpha'][sc_name] * fx(sc)[2:-1, 2:-2, 2:-2] /
        params['dx'])
    f_diff_y = (
        rho_y * params['alpha'][sc_name] * fy(sc)[2:-2, 2:-1, 2:-2] /
        params['dy'])
    f_diff_z = (
        rho_z * params['alpha'][sc_name] * fz(sc)[2:-2, 2:-2, 2:-1] /
        params['dz'])

    diff_x = grad_f(f_diff_x, params['dx'], 'x')
    diff_y = grad_f(f_diff_y, params['dy'], 'y')
    diff_z = grad_f(f_diff_z, params['dz'], 'z')

    rhs_sc = -(conv_x + conv_y + conv_z) + (diff_x + diff_y + diff_z)

    return {
        'rho_{}'.format(sc_name):
            tf.pad(rhs_sc, ((2, 2), (2, 2), (2, 2)), 'CONSTANT')
    }
  def build_jacobian(rho, sc_name, params):
    """Computes the Jacobian for the scalar equations."""
    ndim = rho.get_shape().as_list()
    nbc = params['nbc']
    # permute direction (x, y, z), (y, z, x)
    perm = [[0, 1, 2], [1, 2, 0]]
    perm_end = [[0, 1, 2, 3], [3, 0, 1, 2]]
    grid = [params['dx'], params['dy']]
    # Consider x, y directions for now
    a_diff = [None] * (len(ndim) - 1)
    for i in range(len(ndim) - 1):
      # Permutate the derivative direction to the first direction
      rho_t = tf.transpose(rho, perm=perm[i])
      drho = 0.5 * (rho_t[1:-2, 2:-2, 2:-2] + rho_t[2:-1, 2:-2, 2:-2])
      buf = 0.5 * params['dt'] * drho * params['alpha'][sc_name] / grid[i]**2 
      rho_p = rho_t[nbc:-nbc, nbc:-nbc, nbc:-nbc]
      # calculate the coefficients of the scalar in the diffusion term
      diagonals_0 = tf.pad(buf[1:], ([nbc, 0], [0, 0], [0, 0]),
                           'constant')/tf.pad(rho_p, ([nbc, 0], [0, 0], [0, 0]),
                                              'constant', constant_values=1.)
      diagonals_1 = tf.pad(-(buf[1:] + buf[:-1]), ([1, 1], [0, 0], [0, 0]),
                           'constant')/tf.pad(rho_p, ([1, 1], [0, 0], [0, 0]), 
                                              'constant', constant_values=1.)
      diagonals_2 = tf.pad(buf[:-1], ([0, nbc], [0, 0], [0, 0]),
                           'constant')/tf.pad(rho_p, ([0, nbc], [0, 0], [0, 0]),
                                              'constant', constant_values=1.)
      diagonals = tf.stack([tf.transpose(diagonals_0),
       tf.transpose(diagonals_1), tf.transpose(diagonals_2)], axis = -1)
      diagonals = tf.transpose(diagonals, perm=[0, 1, 3, 2])
      # ensemble the tridiagonal Jacobian matrices
      a_diff[i] = tf.linalg.diag(diagonals, k=(-1, 1))
      a_diff[i] = tf.transpose(a_diff[i], perm=[2, 3, 1, 0])
      a_diff[i] = tf.pad(a_diff[i], ([1,1], [1, 1], [nbc, nbc], [nbc, nbc]),
                         'constant')
      # Permuate back to the original directions
      a_diff[i] = tf.transpose(a_diff[i], perm=perm_end[i])
    return {sc_name : a_diff}

  sc_names = params['sc_names']

  # Update scalars with intermediate density.
  state_m = {}
  for key in sc_names:
    state_m.update({key: 0.5 * (state_old[key] + state[key])})
  state_m.update({'rho': 0.5 * (state_old['rho'] + state['rho'])})

  rho_sc_old = {}
  rho_sc_updates = {}
  jacobian = {}
  for sc_name in sc_names:
    rho_sc_old.update(
        {'rho_{}'.format(sc_name): state_old['rho'] * state_old[sc_name]})

    rho_sc_updates.update(
        rhs(state_m['rho'], state['rhou'], state['rhov'], state['rhow'],
            state_m[sc_name], sc_name))
    if params['add_jacobian']:
      jacobian.update(build_jacobian(state_m['rho'], sc_name, params))

  rho_sc_new = explicit_iterative_time_advancement(rho_sc_old, rho_sc_updates,
                                                   params['dt'])

  sc_tmp = {sc_name: rho_sc_new['rho_{}'.format(sc_name)] / state['rho']
            for sc_name in sc_names}

  if params['add_jacobian']:
    # Solve Ax = b system
    sc_tmp = solve_scalar(rho_sc_new, state, jacobian, params)

  # Update the density with the latest scalars.
  for sc_name in sc_names:
    sc_tmp.update({sc_name: update_boundary(sc_tmp[sc_name], params['nbc'])}) 
  rho_new = density(sc_tmp, params)

  # Update the scalars with the latest density.
  new_rho_scalar = {'rho': rho_new}
  for sc_name in sc_names:
    sc_buf = rho_sc_new['rho_{}'.format(sc_name)] / rho_new
    new_rho_scalar.update({sc_name: update_boundary(sc_buf, params['nbc'])})

  return new_rho_scalar

In [ ]:
#@title Pressure
_NP_DTYPE = np.float32

def make_banded_matrix(stencil, banded_matrix_size, offset=None):
  """Creates a banded matrix.

  The band diagonal elements are populated with the `stencil`.

  Args:
    stencil: List of coefficients in the diagonal band.
    banded_matrix_size: The integer size of the banded matrix.
    offset: Index of the element of the stencil at which to start,
      such that the top-first element of the banded matrix is
      `stencil[offset]`. Defaults to the middle element of the stencil.
  Returns:
    A banded matrix numpy array with shape
    (banded_matrix_size, banded_matrix_size) and type _NP_DTYPE.
  """
  offset = len(stencil) // 2 if offset is None else offset

  stencil = np.asarray(stencil, dtype=_NP_DTYPE)
  padding = np.zeros(banded_matrix_size - 1, dtype=_NP_DTYPE)
  padded_stencil = np.concatenate((padding, stencil, padding))
  strides = padded_stencil.strides[0]
  strided = np.lib.stride_tricks.as_strided
  return strided(padded_stencil[banded_matrix_size - 1 + offset:],
                 shape=(banded_matrix_size, banded_matrix_size),
                 strides=(-strides, strides))


def conjugate_gradient(A, b, x=None, tol=1e-5, maxiter=None):
  x = tf.zeros_like(b) if x is None else x
  maxiter = np.prod(b.get_shape().as_list()) if maxiter is None else maxiter

  r = b - A(x)
  p = r
  rsold = tf.reduce_sum(r**2)

  states0 = {'p': p, 'rsold': rsold, 'x': x, 'r': r}

  cond = lambda i, states: tf.sqrt(states['rsold']) >= tol

  def body(i, states):
    p = states['p']
    rsold = states['rsold']
    x = states['x']
    r = states['r']

    Ap = A(p)
    alpha = rsold / tf.reduce_sum(p * Ap)
    x = x + alpha * p
    r = r - alpha * Ap
    rsnew = tf.reduce_sum(r**2)
    p = r + (rsnew / rsold) * p
    rsold = rsnew

    return i + 1, {'p': p, 'rsold': rsold, 'x': x, 'r': r}

  i0 = tf.constant(0)
  _, new_states = tf.while_loop(cond, body, (i0, states0),
                                back_prop=False, maximum_iterations=maxiter,
                                return_same_structure=True)

  return new_states['x']


def conjugate_gradient_solver(rhs, params):

  def kron(x, a, b, c):
    return (tf.einsum('ip,pjk->ijk', a, x) +
            tf.einsum('jq,iqk->ijk', b, x) +
            tf.einsum('kr,ijr->ijk', c, x))

  ndim = rhs.get_shape().as_list()

  a = [None] * 3
  for i in range(len(ndim)):
    buf = np.zeros(ndim[i])
    buf[0] = -2.
    buf[1] = 1.
    buf[-1] = 1.
    a[i] = sp.linalg.circulant(buf)
    # Commented out the following lines to make a 0 Dirichlet boundary condition
    # out of the boundary layer so the matrix is positive definite.
    # a[i][0, 0] = -1.
    # a[i][-1, -1] = -1.
    a[i][0, -1] = 0
    a[i][-1, 0] = 0
    a[i] = tf.constant(a[i], dtype=tf.float32)

  a[0] /= -params['dx']**2
  a[1] /= -params['dy']**2
  a[2] /= -params['dz']**2

  return conjugate_gradient(functools.partial(kron, a=a[0], b=a[1], c=a[2]),
                            - rhs + tf.reduce_mean(rhs), tol=1e-3,
                            maxiter=np.prod(rhs.get_shape().as_list()))

def fast_diagonalization_solver(rhs, params):
  
  def kron(x, a, b, c):
    return tf.einsum('ip,jq,kr,pqr->ijk', a, b, c, x)

  ndim = rhs.get_shape().as_list()

  a = [None] * 3
  for i in range(len(ndim)):
    buf = np.zeros(ndim[i])
    buf[0] = -2.
    buf[1] = 1.
    buf[-1] = 1.
    a[i] = sp.linalg.circulant(buf)
    a[i][0, 0] = -1.
    a[i][-1, -1] = -1.
    a[i][0, -1] = 0
    a[i][-1, 0] = 0

  a[0] /= params['dx']**2
  a[1] /= params['dy']**2
  a[2] /= params['dz']**2

  lam = [None] * 3
  v = [None] * 3
  for i in range(3):
    lam_i, v_i = sp.linalg.eigh(a[i])
    v[i] = np.array(v_i)
    if i == 0:
      lam[i] = np.reshape(lam_i, (-1, 1, 1))
    elif i == 1:
      lam[i] = np.reshape(lam_i, (1, -1, 1))
    else:  # i == 2
      lam[i] = np.reshape(lam_i, (1, 1, -1))
  lam_sum = tf.constant(lam[0] + lam[1] + lam[2], dtype=tf.float32)
  lam_inv = tf.where(tf.less(tf.abs(lam_sum), 1e-8),
                     tf.zeros_like(lam_sum, dtype=tf.float32),
                     1. / lam_sum)

  # Step 1.
  buf = kron(rhs - tf.reduce_mean(rhs),
             tf.constant(v[0].transpose(), dtype=tf.float32), 
             tf.constant(v[1].transpose(), dtype=tf.float32), 
             tf.constant(v[2].transpose(), dtype=tf.float32))

  # Step 2.
  buf = lam_inv * buf

  # Step 3.
  return kron(buf,
              tf.constant(v[0], dtype=tf.float32), 
              tf.constant(v[1], dtype=tf.float32), 
              tf.constant(v[2], dtype=tf.float32))


def jacobi_solver(rhs, params):
  dx = params['dx']
  dy = params['dy']
  dz = params['dz']
  nbc = params['nbc']
  ndim = rhs.get_shape().as_list()
  nit = 10
  omega = 1.0

  s = [tf.constant(make_banded_matrix([1., 0., 1.], n), dtype=tf.float32)
   for n in ndim]

  factor_x = (1. + (dx/dy)**2 + (dx/dz)**2)
  factor_y = (1. + (dy/dx)**2 + (dy/dz)**2)
  factor_z = (1. + (dz/dx)**2 + (dz/dy)**2)

  rhs_0 = rhs - tf.reduce_mean(rhs)

  def body(i, f):
    f = update_boundary(f, nbc) 

    f_interior_buf = (0.5 * tf.einsum('ip,pqr->iqr', s[0], f) / factor_x + 
                      0.5 * tf.einsum('jq,pqr->pjr', s[1], f) / factor_y +
                      0.5 * tf.einsum('kr,pqr->pqk', s[2], f) / factor_z -
                      (dx**2 / (6. * factor_x) + 
                       dy**2 / (6. * factor_y) +
                       dz**2 / (6. * factor_z)) * rhs)
    f_interior = f_interior_buf[1:-1, 1:-1, 1:-1]
    f_x = tf.concat([tf.expand_dims(f_interior[0, :, :], 0), 
                     f_interior, 
                     tf.expand_dims(f_interior[-1, :, :], 0)], 0)
    f_y = tf.concat([tf.expand_dims(f_x[:, 0, :], 1),
                     f_x, 
                     tf.expand_dims(f_x[:, -1, :], 1)], 1)
    f_new = tf.concat([tf.expand_dims(f_y[:, :, 0], 2), 
                       f_y, 
                       tf.expand_dims(f_y[:, :, -1], 2)], 2)
    return i + 1, omega * f_new + (1.0 - omega) * f

  stop_condition = lambda i, f: i < nit
  
  i0 = tf.constant(0)
  f0 = tf.zeros_like(rhs, dtype=tf.float32)

  _, f = tf.while_loop(
      stop_condition,
      body,
      loop_vars=(i0, f0),
      back_prop=False,
      return_same_structure=True
  )

  return update_boundary(f, nbc)


def pressure(state_old, state, params):
  # Update the pressure correction term.
  d4p_dx4 = apply_kernel(state['p'], 'x', 'd4x2')
  d4p_dy4 = apply_kernel(state['p'], 'y', 'd4x2')
  d4p_dz4 = apply_kernel(state['p'], 'z', 'd4x2')

  coeff_x = params['dt'] / (4. * params['dx']**2)
  coeff_y = params['dt'] / (4. * params['dy']**2)
  coeff_z = params['dt'] / (4. * params['dz']**2)

  drhou_dx = (grad(state['rhou'], params['dx'], 'x', 'ddx2') + 
              coeff_x * d4p_dx4)
  drhov_dy = (grad(state['rhov'], params['dy'], 'y', 'ddx2') + 
              coeff_y * d4p_dy4)
  drhow_dz = (grad(state['rhow'], params['dz'], 'z', 'ddx2') + 
              coeff_z * d4p_dz4)

  pressure_rhs = 1. / params['dt'] * (
    drhou_dx + drhov_dy + drhow_dz + 
    drho(state['rho'], state_old['rho'], 1) / params['dt']
  )

  # dp = fast_diagonalization_solver(update_boundary(pressure_rhs, params['nbc']),
  #                                  params)
  dp = jacobi_solver(update_boundary(pressure_rhs, params['nbc']), params)
  # dp = conjugate_gradient_solver(update_boundary(pressure_rhs, params['nbc']),
  #                                params)

  return dp - tf.reduce_mean(dp[params['nbc']:-params['nbc'],
                                params['nbc']:-params['nbc'],
                                params['nbc']:-params['nbc']])

## Simulation Procedure

In [ ]:
def prestep(state_old, state):
  """Prepares the initial guess at the begining of the subiteration."""
  for varname, _ in state_old.items():
    if varname == 'rho':
      rho_old = state_old['rho']
      state_old['rho'] = state['rho']
      state['rho'] = 2. * state['rho'] - rho_old
    else:
      state_old[varname] = state[varname]

  return state_old, state

In [ ]:
def poststep(state_old, state, dp, params):
  """Update the velocity and pressure at the end of each subiteration."""
  rho_m = 0.5 * (state_old['rho'] + state['rho'])

  rhou = state['rhou'] - params['dt'] * grad(dp, params['dx'], 'x', 'ddx2')
  rhov = state['rhov'] - params['dt'] * grad(dp, params['dy'], 'y', 'ddx2')
  rhow = state['rhow'] - params['dt'] * grad(dp, params['dz'], 'z', 'ddx2')

  state_new = {
      'u': update_boundary(rhou / rho_m, params['nbc']),
      'v': update_boundary(rhov / rho_m, params['nbc']),
      'w': update_boundary(rhow / rho_m, params['nbc']),
      'p': update_boundary(state['p'] + dp, params['nbc']),
      'rho': state['rho'],
  }
  state_new.update({
      'rhou': state_new['u'] * rho_m,
      'rhov': state_new['v'] * rho_m,
      'rhow': state_new['w'] * rho_m,
  })
  state_new.update({sc_name: state[sc_name] for sc_name in params['sc_names']})

  return state_new

In [ ]:
def subiteration(state_old, state, params):
  nit = 8

  # Step 1.
  state_old, state = prestep(state_old, state)

  def body(i, state):
    # Step 2 - 4.
    density_scalar = rho_scalar(state_old, state, params)
    for varname, value in density_scalar.items():
      state[varname] = value

    # Step 5.
    velocity = momentum(state_old, state, params)
    for varname, value in velocity.items():
      state[varname] = value

    # Step 6.
    dp = pressure(state_old, state, params)

    # Step 7.
    state = poststep(state_old, state, dp, params)

    return (i + 1, state)

  stop_condition = lambda i, state: i < nit

  i0 = tf.constant(0)
  _, state_new = tf.while_loop(
      stop_condition,
      body,
      loop_vars=(i0, state),
      return_same_structure=True,
      back_prop=False)

  return state_new

In [ ]:
def run_simulation(state, params):
  
  def body(i, state):
    # Save the old states at the beginning of each iteration.
    state_old = state

    state = subiteration(state_old, state, params)

    return (i + 1, state)

  stop_condition = lambda i, state: i < params['nit']

  i0 = tf.constant(0)
  _, state_new = tf.while_loop(
      stop_condition,
      body,
      loop_vars=(i0, state),
      return_same_structure=True,
      back_prop=False)

  return state_new


# Run Simulation

## Initial Condition

In [ ]:
def tf_setup_grid(nx, ny, nz, lx, ly, lz, nbc, option):

  def tf_convection():
    """Linear convection in one direction."""
    r = 30.
    u_l = np.tanh(r * (Y - 0.25)) + 1.0
    u_r = np.tanh(r * (0.75 - Y)) + 1.0
    u = u_l
    u[Y > 0.5] = u_r[Y > 0.5]
    u = tf.convert_to_tensor(u, dtype=tf.float32)

    return {
        'u': u,
        'v': tf.zeros_like(u, dtype=tf.float32),
        'w': tf.zeros_like(u, dtype=tf.float32),
        'rhou': u,
        'rhov': tf.zeros_like(u, dtype=tf.float32),
        'rhow': tf.zeros_like(u, dtype=tf.float32),
        'p': tf.zeros_like(u, dtype=tf.float32),
        'rho': tf.ones_like(u, dtype=tf.float32),
        'Y1': tf.zeros_like(u, dtype=tf.float32),
    }

  def tf_vortex_rollup():
    """
    2D Vortex Rollup Navier-Stokes example.

    u = tanh(ρ (y - 0.25))     y ≤ 0.5
        tanh(ρ (0.75 - y))     y > 0.5
    v = 0.05 sin (2 π x)
    """
    v_field = np.zeros((3, nx, ny, nz), dtype=np.float32)                                
    r = 30.
    u_l = np.tanh(r * (Y - 0.25))
    u_r = np.tanh(r * (0.75 - Y))
    u = u_l
    u[Y > 0.5] = u_r[Y > 0.5]
    v_field[0, :, :, :] = u
    v_field[1, :, :, :] = 0.05 * np.sin(2. * np.pi * X)
    p = tf.zeros(shape=[nx, ny, nz], dtype=tf.float32)

    sc_field = {}
    sc_l = 0.5 * (np.tanh(r * (Y - 0.25)) + 1.)
    sc_u = 0.5 * (np.tanh(r * (0.75 - Y)) + 1.)
    sc_0 = sc_l
    sc_0[Y > 0.5] = sc_u[Y > 0.5]
    # sc_0 = np.sin(2. * np.pi * X)
    sc_field.update({'Y1': sc_0})

    rho_0 = 0.5
    rho_1 = 1.5
    rho = rho_0 * sc_field['Y1'] + rho_1 * (1. - sc_field['Y1'])

    state_0 = {
        'u': tf.constant(v_field[0, :, :, :], dtype=tf.float32),
        'v': tf.constant(v_field[1, :, :, :], dtype=tf.float32),
        'w': tf.constant(v_field[2, :, :, :], dtype=tf.float32),
        'rhou': tf.constant(rho * v_field[0, :, :, :], dtype=tf.float32),
        'rhov': tf.constant(rho * v_field[1, :, :, :], dtype=tf.float32),
        'rhow': tf.constant(rho * v_field[2, :, :, :], dtype=tf.float32),
        'rho': tf.constant(rho, dtype=tf.float32),
        'p': p
    }
    state_0.update({key: tf.constant(value, dtype=tf.float32) for 
                    key, value in sc_field.items()})
    return state_0

  def tf_taylor_green_vortex():
    """
    2D Taylor-Green Vortex example.

    u = sin(2 pi x)cos(2 pi y)exp(-8 pi^2 mu t)
    v = -cos(2 pi x)sin(2 pi y)exp(-8 pi^2 mu t)
    p = 0.25 [cos(4 pi x) + cos(4 pi y)]exp(-16 pi^2 mu t)
    """
    v_field = np.zeros((3, nx, ny, nz), dtype=np.float32)
    u = np.sin(2. * np.pi * X) * np.cos(2. * np.pi * Y)
    v = - np.cos(2. * np.pi * X) * np.sin(2. * np.pi * Y)
    v_field[0, :, :, :] = u
    v_field[1, :, :, :] = v
    p = 0.25 * (np.cos(4 * np.pi * X) + np.cos(4 * np.pi * Y))
    p = tf.constant(p, dtype=tf.float32)
    rho = 1.
    state_0 = {
        'u': tf.constant(v_field[0, :, :, :], dtype=tf.float32),
        'v': tf.constant(v_field[1, :, :, :], dtype=tf.float32),
        'w': tf.constant(v_field[2, :, :, :], dtype=tf.float32),
        'rhou': tf.constant(rho * v_field[0, :, :, :], dtype=tf.float32),
        'rhov': tf.constant(rho * v_field[1, :, :, :], dtype=tf.float32),
        'rhow': tf.constant(rho * v_field[2, :, :, :], dtype=tf.float32),
        'rho': tf.ones_like(p, dtype=tf.float32),
        'p': p,
        'Y1': 0.5 * tf.ones_like(p, dtype=tf.float32)
    }
    return state_0

  nx -= 2 * nbc
  x = np.linspace(0., lx, nx + 1)
  x = x[:-1]
  dx = x[1] - x[0]

  ny -= 2 * nbc
  y = np.linspace(0., ly, ny + 1)
  y = y[:-1]
  dy = y[1] - y[0]

  nz -= 2 * nbc
  z = np.linspace(0., lz, nz + 1)
  z = z[:-1]
  dz = z[1] - z[0]

  X, Y, Z = np.meshgrid(x, y, z, indexing='ij')

  if option == 'vortex_rollup':
    state_0 = tf_vortex_rollup()
  elif option == 'convection':
    state_0 = tf_convection()
  elif option == 'taylor_green_vortex':
    state_0 = tf_taylor_green_vortex()
  else:
    raise ValueError('{} is not a valid initial configuration.'.format(option))

  state_params = {
      'x': X, 'dx': dx,
      'y': Y, 'dy': dy, 
      'z': Z, 'dz': dz, 
  }
  state_params.update({
      varname: 
      update_boundary(
          tf.pad(value, ((nbc, nbc), (nbc, nbc), (nbc, nbc)), 'CONSTANT'), nbc) 
      for varname, value in state_0.items()})

  return state_params
          

## Run and Postprocess

In [ ]:
#@title Parameters
dt = 1e-3  #@param {type: "number"}
nx = 128  #@param {type: "number"}
ny = 128  #@param {type: "number"}
nz = 6  #@param {type: "number"}
nt = 500  #@param {type: "number"}
nit = 50  #@param {type: "number"}
nbc = 2  #@param {type: "number"}
lx = 1.  #@param {type: "number"}
ly = 1.  #@param {type: "number"}
lz = 0.05  #@param {type: "number"}
add_jacobian = True #@param {type: "boolean"}
init_flow = True  #@param {type: "boolean"}
initial_configuration = "taylor_green_vortex"  #@param ["vortex_rollup", "convection", "taylor_green_vortex"]
debug = False  #@param {type: "boolean"}

In [ ]:
#@title Simulation Run
#@test {"output": "screenshot"}
#@test {"timeout": 60}
if init_flow:
  init_states = tf_setup_grid(nx, ny, nz, lx, ly, lz, nbc, 
                              initial_configuration)

  state_0 = {varname: value 
             for varname, value in init_states.items() 
             if varname not in ('x', 'y', 'z', 'dx', 'dy', 'dz')}

  state_var = {key: tf.Variable(value) for key, value in state_0.items()}
else:
  state_var = {key: tf.Variable(value) for key, value in state.items()}

params = {'dt': dt,
          'nit': nit,
          'nbc': nbc, 
          'dx': init_states['dx'],
          'dy': init_states['dy'],
          'dz': init_states['dz'],
          'nu': 2e-3,
          'alpha': {'Y1': 2e-3},
          'rho': {'Y1': 0.5, 'YN': 1.5},
          'sc_names': ('Y1',),
          'add_jacobian': add_jacobian,
          'dbg': debug}

state_graph = run_simulation(state_var, params)
state_update = {key: tf.assign(state_var[key], value) for 
                key, value in state_graph.items()}

with tf.Session() as sess:
  with sess.as_default():
    tf.initializers.global_variables().run()

    print('Iter\t Min. U\t Max. U\t Min. V\t Max. V\t '
    'Min. W\t Max. W\t Min. P\t Max. P\t')
    for i in range(nt // nit):
      state = sess.run(state_update)

      print('%i %5.3e %5.3e %5.3e %5.3e %5.3e %5.3e %5.3e %5.3e' % (
        (i + 1) * nit,
        np.min(state['u']), np.max(state['u']),
        np.min(state['v']), np.max(state['v']),
        np.min(state['w']), np.max(state['w']),
        np.min(state['p']), np.max(state['p'])))


In [ ]:
#@title Visualization and Postprocessing
# Visualization of the vortex rollup simulation.
slice = 1
n = 10
t = n * params['dt']

u0 = np.reshape(state['u'][:, -1, :], (nx, 1, nz))
u1 = np.reshape(state['u'][:,  0, :], (nx, 1, nz))
u = np.concatenate((u0, state['u'], u1), axis=1)
v0 = np.reshape(state['v'][-1, :, :], (1, ny, nz))
v1 = np.reshape(state['v'][ 0, :, :], (1, ny, nz))
v = np.concatenate((v0, state['v'], v1), axis=0)
vorticity = ((v[2:, :, :] - v[:-2, :, :]) / 2. / params['dx'] - 
             (u[:, 2:, :] - u[:, :-2, :]) / 2. / params['dy'])
state['vorticity'] = vorticity
varnames = ['Y1', 'rho', 'u', 'v', 'vorticity'] #@param
var = [state[name] for name in varnames]
plt.figure(figsize=(36, 4))
for i in range(len(var)):
  plt.subplot(1, len(var), i + 1)
  plt.contourf(
      var[i][:, :, 3],
      cmap='jet',
      levels=np.linspace(np.min(var[i]), np.max(var[i]), 21))
  plt.colorbar()

In [ ]:
#@title Error Analysis
# Calculate L infinity error for taylor_green_vortex config
def err_analysis(params, init_states, state, t):
  mu = params['nu']
  X = init_states['x']
  Y = init_states['y']
  Z = init_states['z']
  u = np.sin(2. * np.pi * X) * np.cos(2. * np.pi * Y) * np.exp(
      -8 * np.pi**2 * mu * t)
  v = -np.cos(2. * np.pi * X) * np.sin(2. * np.pi * Y) * np.exp(
      -8 * np.pi**2 * mu * t)
  p = 0.25 * (np.cos(4 * np.pi * X) + np.cos(4 * np.pi * Y)) * np.exp(
      -16 * np.pi**2 * mu * t)
  y1 = 0.5 * np.ones_like(X)
  u_diff = np.absolute(u - state['u'][2:-2, 2:-2, 2:-2])
  v_diff = np.absolute(v - state['v'][2:-2, 2:-2, 2:-2])
  p_diff = np.absolute(p - state['p'][2:-2, 2:-2, 2:-2])
  y1_diff = np.absolute(y1 - state['Y1'][2:-2, 2:-2, 2:-2])
  u_err = np.max(u_diff)
  v_err = np.max(v_diff)
  p_err = np.max(p_diff)
  y1_err = np.max(y1_diff)
  print(u_err)
  print(v_err)
  print(p_err)
  print(y1_err)


if initial_configuration == 'taylor_green_vortex':
  err_analysis(params, init_states, state, dt * nt)